In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

### upload year 2010 daily data

In [4]:
ur_files = ddf.read_csv('DailyTemp/DailyTemp_2009.csv', dtype = {'fips': str, 'STATION': 'object'})
temp = ur_files.compute()
temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
temp['year'] = '2009'
temp.head()

,STATION,LATITUDE,LONGITUDE,NAME,YEAR,DATE,TEMP,MIN,MAX,year
0,826099999,0.0,0.0,WXPOD8270,2009.0,2009-01-01,29.3,14.0,44.6,2009
1,826099999,0.0,0.0,WXPOD8270,2009.0,2009-01-02,42.4,14.0,55.9,2009
2,826099999,0.0,0.0,WXPOD8270,2009.0,2009-01-03,42.7,26.6,62.6,2009
3,826099999,0.0,0.0,WXPOD8270,2009.0,2009-01-04,38.3,21.2,62.6,2009
4,826099999,0.0,0.0,WXPOD8270,2009.0,2009-01-05,35.0,23.0,48.9,2009


In [5]:
temp['YEAR'].unique()

array([2009., 2018., 2019.,   nan])

In [6]:
def rangeBelow30(temp):
    if temp < 30:
        return 1
    else:
        return 0

In [7]:
def range30To40(temp):
    if temp < 40 and temp >= 30:
        return 1
    else:
        return 0

In [8]:
def range40To50(temp):
    if temp < 50 and temp >= 40:
        return 1
    else:
        return 0

In [9]:
def range50To60(temp):
    if temp < 60 and temp >= 50:
        return 1
    else:
        return 0

In [10]:
def range60To70(temp):
    if temp < 70 and temp >= 60:
        return 1
    else:
        return 0

In [11]:
def range70To80(temp):
    if temp < 80 and temp >= 70:
        return 1
    else:
        return 0

In [12]:
def range80To90(temp):
    if temp < 90 and temp >= 80:
        return 1
    else:
        return 0

In [13]:
def rangeOver90(temp):
    if temp >= 90:
        return 1
    else:
        return 0

In [14]:
def changeToNone(value):
    if value == 999.9 or value == 9999.9 or value == 99.99 or value == 99.9:
        return np.nan
    else:
        return value

In [15]:
temp['TEMP'] = temp['TEMP'].apply(changeToNone)

In [16]:
temp['<30'] = temp['TEMP'].apply(rangeBelow30)
temp['30-40'] = temp['TEMP'].apply(range30To40)
temp['40-50'] = temp['TEMP'].apply(range40To50)
temp['50-60'] = temp['TEMP'].apply(range50To60)
temp['60-70'] = temp['TEMP'].apply(range60To70)
temp['70-80'] = temp['TEMP'].apply(range70To80)
temp['80-90'] = temp['TEMP'].apply(range80To90)
temp['>=90'] = temp['TEMP'].apply(rangeOver90)

In [17]:
temp.head()

,STATION,LATITUDE,LONGITUDE,NAME,YEAR,DATE,TEMP,MIN,MAX,year,<30,30-40,40-50,50-60,60-70,70-80,80-90,>=90
0,826099999,0.0,0.0,WXPOD8270,2009.0,2009-01-01,29.3,14.0,44.6,2009,1,0,0,0,0,0,0,0
1,826099999,0.0,0.0,WXPOD8270,2009.0,2009-01-02,42.4,14.0,55.9,2009,0,0,1,0,0,0,0,0
2,826099999,0.0,0.0,WXPOD8270,2009.0,2009-01-03,42.7,26.6,62.6,2009,0,0,1,0,0,0,0,0
3,826099999,0.0,0.0,WXPOD8270,2009.0,2009-01-04,38.3,21.2,62.6,2009,0,1,0,0,0,0,0,0
4,826099999,0.0,0.0,WXPOD8270,2009.0,2009-01-05,35.0,23.0,48.9,2009,0,1,0,0,0,0,0,0


In [18]:
temp = temp.groupby(['STATION', 'LATITUDE', 'LONGITUDE']).sum()
temp = temp.reset_index()
temp.head()

,STATION,LATITUDE,LONGITUDE,YEAR,TEMP,MIN,MAX,<30,30-40,40-50,50-60,60-70,70-80,80-90,>=90
0,01183099999,68.491300,16.678108,731276.0,14224.0,12049.4,15953.6,100,90,84,66,22,2,0,0
1,06616099999,47.433333,6.950000,363629.0,9573.6,8284.6,10968.5,8,24,38,50,49,12,0,0
2,07549099999,44.891388,2.421944,733285.0,18493.2,14916.7,22672.4,18,67,88,89,84,19,0,0
3,10000199999,51.602414,6.142172,731276.0,19019.9,16049.0,21485.0,17,46,80,107,88,26,0,0
4,1001099999,70.933333,-8.666667,733285.0,12207.8,10991.3,13306.4,111,172,81,1,0,0,0,0


In [19]:
temp = temp[['STATION', 'LATITUDE', 'LONGITUDE', '<30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '>=90']]
temp['year'] = '2009'
temp = temp.rename(columns = {'STATION': 'station', 'LATITUDE': 'latitude', 'LONGITUDE': 'longitude'})
temp.head()

,station,latitude,longitude,<30,30-40,40-50,50-60,60-70,70-80,80-90,>=90,year
0,01183099999,68.491300,16.678108,100,90,84,66,22,2,0,0,2009
1,06616099999,47.433333,6.950000,8,24,38,50,49,12,0,0,2009
2,07549099999,44.891388,2.421944,18,67,88,89,84,19,0,0,2009
3,10000199999,51.602414,6.142172,17,46,80,107,88,26,0,0,2009
4,1001099999,70.933333,-8.666667,111,172,81,1,0,0,0,0,2009


In [20]:
temp.to_csv('TempCount/Temp_Count_2009.csv')

In [20]:
years = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

In [21]:
for year in years:
    ur_files = ddf.read_csv('DailyTemp_' + year + '.csv', dtype = {'STATION': 'object'})
    temp = ur_files.compute()
    temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
    temp['TEMP'] = temp['TEMP'].apply(changeToNone)
    temp['<30'] = temp['TEMP'].apply(rangeBelow30)
    temp['30-40'] = temp['TEMP'].apply(range30To40)
    temp['40-50'] = temp['TEMP'].apply(range40To50)
    temp['50-60'] = temp['TEMP'].apply(range50To60)
    temp['60-70'] = temp['TEMP'].apply(range60To70)
    temp['70-80'] = temp['TEMP'].apply(range70To80)
    temp['80-90'] = temp['TEMP'].apply(range80To90)
    temp['>=90'] = temp['TEMP'].apply(rangeOver90)
    temp = temp.groupby(['STATION', 'LATITUDE', 'LONGITUDE']).sum()
    temp = temp.reset_index()
    temp = temp[['STATION', 'LATITUDE', 'LONGITUDE', '<30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '>=90']]
    temp['year'] = year
    temp = temp.rename(columns = {'STATION': 'station', 'LATITUDE': 'latitude', 'LONGITUDE': 'longitude'})
    temp.to_csv('Temp_Count_' + year + '.csv')